In [1]:
import datajoint as dj
import os
from pathlib import Path

# set dirs
base_dir = Path('/Users/rly/Documents/NWB/spyglass-workspace') # change this to your desired directory
# base_dir = Path('/hdd/dj') # change this to your desired directory
if (base_dir).exists() is False:
    os.mkdir(base_dir)
raw_dir = base_dir / 'raw'
if (raw_dir).exists() is False:
    os.mkdir(raw_dir)
analysis_dir = base_dir / 'analysis'
if (analysis_dir).exists() is False:
    os.mkdir(analysis_dir)
recording_dir = base_dir / 'recording'
if (recording_dir).exists() is False:
    os.mkdir(recording_dir)
sorting_dir = base_dir / 'sorting'
if (sorting_dir).exists() is False:
    os.mkdir(sorting_dir)
waveforms_dir = base_dir / 'waveforms'
if (waveforms_dir).exists() is False:
    os.mkdir(waveforms_dir)
tmp_dir = base_dir / 'tmp'
if (tmp_dir).exists() is False:
    os.mkdir(tmp_dir)
kachery_cloud_dir = base_dir / '.kachery_cloud'
if (kachery_cloud_dir).exists() is False:
    os.mkdir(kachery_cloud_dir)

# set dj config
dj.config['database.host'] = 'localhost'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'tutorial'
dj.config['database.port'] = 3306
dj.config['stores'] = {
  'raw': {
    'protocol': 'file',
    'location': str(raw_dir),
    'stage': str(raw_dir)
  },
  'analysis': {
    'protocol': 'file',
    'location': str(analysis_dir),
    'stage': str(analysis_dir)
  }
}

# set env vars
os.environ['SPYGLASS_BASE_DIR'] = str(base_dir)
os.environ['SPYGLASS_RECORDING_DIR'] = str(recording_dir)
os.environ['SPYGLASS_SORTING_DIR'] = str(sorting_dir)
os.environ['SPYGLASS_WAVEFORMS_DIR'] = str(waveforms_dir)
os.environ['SPYGLASS_TEMP_DIR'] = str(tmp_dir)
os.environ['KACHERY_CLOUD_DIR'] = str(kachery_cloud_dir)

os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = 'TRUE'

dj.config["enable_python_native_blobs"] = True

import spyglass as sg
import spyglass.common as sgc
import spyglass.data_import as sgdi
import spyglass.spikesorting as sgs
import warnings
warnings.simplefilter("always")

[2022-10-20 16:52:55,882][INFO]: Connecting root@localhost:3306
[2022-10-20 16:52:56,048][INFO]: Connected root@localhost:3306
/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/position_tools/core.py:3: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
# sgc.DataAcquisitionDeviceSystem().delete(safemode=False)
# sgc.DataAcquisitionDeviceAmplifier().delete(safemode=False)
# sgc.DataAcquisitionDevice().delete(safemode=False)
# sgc.Institution().delete(safemode=False, force_parts=True)
# sgc.Lab().delete(safemode=False)
# sgc.LabTeam().delete(safemode=False)
# sgc.LabMember().delete(safemode=False)
# sgc.Subject().delete(safemode=False)
sgc.Session().delete(safemode=False)
sgc.Nwbfile().delete(safemode=False)
sgc.Nwbfile().cleanup(delete_files=True)
# sgc.Probe().delete(safemode=False)
sgs.SortGroup().delete(safemode=False)
sgs.SortInterval().delete(safemode=False)
sgs.SpikeSorterParameter().delete(safemode=False)
sgs.SpikeSortingPreprocessingParameter().delete(safemode=False)
sgs.SpikeSortingRecordingSelection().delete(safemode=False)
sgs.ImportedSpikeSorting().delete(safemode=False, force_parts=True)
sgs.Curation().delete(safemode=False)


[2022-10-20 16:52:58,386][INFO]: Deleting 1 rows from `common_ephys`.`_raw`
[2022-10-20 16:52:58,410][INFO]: Deleting 1 rows from `common_interval`.`interval_list`
[2022-10-20 16:52:58,443][INFO]: Deleting 1 rows from `common_session`.`_experimenter_list__experimenter`
[2022-10-20 16:52:58,456][INFO]: Deleting 1 rows from `common_session`.`_experimenter_list`
[2022-10-20 16:52:58,471][INFO]: Deleting 1 rows from `common_session`.`_session`
[2022-10-20 16:52:58,490][INFO]: Deleting 1 rows from `common_nwbfile`.`nwbfile`
100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.19it/s]
[2022-10-20 16:52:58,667][INFO]: Deleting 0 rows from `spikesorting_recording`.`sort_group`
[2022-10-20 16:52:58,717][INFO]: Deleting 0 rows from `spikesorting_recording`.`sort_interval`
[2022-10-20 16:52:58,735][INFO]: Deleting 0 rows from `spikesorting_sorting`.`spike_sorter_parameter`
[2022-10-20 16:52:58,754][INFO]: Deleting 0 rows from `spikesorting_recording`

0

In [3]:
sgc.Subject()

subject_id,age,description,genotype,sex,species
AppleBottom,None,None,WT (RRID:IMSR_JAX:000664),F,Mus musculus


In [4]:
# Define the name of the file that you copied and renamed; make sure it's something unique. 
# nwb_file_name = 'sub-CuffLinks_ses-CuffLinks-LT03-g3_behavior+ecephys.nwb'
nwb_file_name = 'sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys.nwb'

filename, file_extension = os.path.splitext(nwb_file_name)
# This is a copy of the original nwb file, except it doesn't contain the raw data (for storage reasons)
nwb_file_name2 = filename + '_' + file_extension

In [5]:
sgdi.insert_sessions(nwb_file_name)

Creating a copy of NWB file sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys.nwb with link to raw ephys data: sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb


/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/rly/Documents/NWB/spyglass-workspace/raw/sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/datajoint/external.py:276: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:


Populate Session...
/Users/rly/Documents/NWB/spyglass-workspace/raw/sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb
Institution...

Lab...

LabMember...
Please add the Google user ID for Emily A. Aery Jones in the LabMember.LabMemberInfo table if you want to give them permission to manually curate sortings by this team.


/Users/rly/Documents/NWB/spyglass/src/spyglass/common/common_lab.py:124: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if not query:



Subject...

DataAcquisitionDevice...
No conforming data acquisition device metadata found.

CameraDevice...
No conforming camera device metadata found.

Probe...
No conforming probe metadata found.

Skipping Apparatus for now...
IntervalList...
No epochs found in NWB file.

Populate ExperimenterList...

Populate Raw...
Importing raw data: Sampling rate:	30000.0 Hz
Number of valid intervals:	1

Populate Electrode...
rel_x value in NWB file (43.0) does not match the rel_x value for the Probe ({probe_electrode_rel_x}). Please check that these values are correct. They may be based off of a different zero or mirror images of each other.
rel_z value in NWB file (3820.0) does not match the rel_z value for the Probe ({probe_electrode_rel_z}). Please check that these values are correct. They may be based off of a different zero or mirror images of each other.

Populate SampleCount...
Unable to import SampleCount: no data interface named "sample_count" found in sub-AppleBottom_ses-AppleBottom-D

In [6]:
sgc.Probe()

probe_type,probe_description description of this probe,num_shanks number of shanks on this device,contact_side_numbering electrode numbers from contact side of the device
Neuropixels 1.0,"{'name': '', 'probe_name': 'Neuropixels 1.0', 'probe_part_number': 'PRB_1_4_0480_1', 'probe_serial_number': '17131307831'}",1,False


In [7]:
sgc.Subject()

subject_id,age,description,genotype,sex,species
AppleBottom,None,None,WT (RRID:IMSR_JAX:000664),F,Mus musculus
